In [118]:
import numpy as np
import pandas as pd
import evaluate
import pickle

from utils import read_jsonl, continuous_string, label_distribution

from transformers import AutoModelForTokenClassification, AutoTokenizer, DataCollatorForTokenClassification, TrainingArguments, Trainer, pipeline, DistilBertTokenizerFast
from huggingface_hub import notebook_login
from datasets import Dataset, DatasetDict

from sklearn.model_selection import train_test_split

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/chris/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [120]:
# Function to read NER data
def read_universal_NER(file_path):
    with open(file_path, 'r', encoding = 'utf-8') as infile:
        # Split into lines
        lines = infile.readlines()

        # Define lists to store data 
        sentences = []
        labels = []
        current_sentence = []
        current_labels = []

        # Iterate over lines
        for line in lines:

            line = line.strip() # Remove whitespace
            if not line: # Skip empty lines
                continue

            # Check if line starts with sentence ID
            if line.startswith('# sent_id'):
                if current_sentence:
                    sentences.append(current_sentence)
                    labels.append(current_labels)
                current_sentence = []
                current_labels = []

            # Check for token lines
            elif not line.startswith("#"):
                parts = line.strip().split('\t')
                current_sentence.append(parts[1])
                current_labels.append(parts[2])

        if current_sentence:
            sentences.append(current_sentence)
            labels.append(current_labels)

    # Flatten lists
    #sentences = sum(sentences, [])
    #labels = sum(labels, [])

    return sentences, labels

In [123]:
# test_sents, test_labels = read_universal_NER('train_dev_test_sets/test.iob2')
# train_sents, train_labels = read_universal_NER('train_dev_test_sets/train.iob2')
# dev_sents, dev_labels = read_universal_NER('train_dev_test_sets/dev.iob2')

with open('train_dev_test_sets/train_raw.pkl', 'rb') as infile:
    train = pickle.load(infile)

with open('train_dev_test_sets/dev_raw.pkl', 'rb') as infile:
    dev = pickle.load(infile)

with open('train_dev_test_sets/test_raw.pkl', 'rb') as infile:
    test = pickle.load(infile)

In [121]:
def format_gold_to_iob2_bert(gold_data):
    tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased') # Define model for tokenization
    output = []
    line_idx = 1

    for document in gold_data:
        text = document['text']
        
        encoded = tokenizer(text, return_offsets_mapping = True) # Tokenize and get offsets
        tokens = tokenizer.convert_ids_to_tokens(encoded.input_ids) # Get token texts
        offsets = encoded.offset_mapping

        labels = sorted(document['label'], key = lambda x: x[0])  # Sort labels by start index

        # Prepare the tags
        tags = ['O'] * len(tokens)

        # Apply IOB tagging
        for start, end, label in labels:
            started = False
            for i, (token_start, token_end) in enumerate(offsets):
                if token_start >= start and token_end <= end:
                    if token_start == start or not started:
                        tags[i] = f'B-{label}'
                        started = True
                    else:
                        tags[i] = f'I-{label}'

        # Generate the formatted output
        for token_idx, token_text in enumerate(tokens):
            # Don't label start and end tokens
            if token_text == '[CLS]' or token_text == '[SEP]':
                output.append(f"{line_idx}\t{token_text}\tO\t-\t-")
            else:
                output.append(f"{line_idx}\t{token_text}\t{tags[token_idx]}\t-\t-")
            line_idx += 1

    output.append('\n')
    return "\n".join(output)

In [124]:
# Convert train, dev, and test to iob2
iob2_sets = {
    'train': format_gold_to_iob2_bert(train),
    'dev': format_gold_to_iob2_bert(dev),
    'test': format_gold_to_iob2_bert(test)
}

# Save sets to disk
for name, iob2_set in iob2_sets.items():
    with open(f'train_dev_test_sets/{name}_for_baseline.iob2', 'w', encoding = 'utf-8') as outfile:
        for line in iob2_set:
            outfile.write(line)

/Users/chris/myenv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (649 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1102 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1037 > 512). Running this sequence through the model will result in indexing errors


In [252]:
train_sents, train_labels = read_universal_NER("train_dev_test_sets/train_for_baseline.iob2")
dev_sents, dev_labels = read_universal_NER("train_dev_test_sets/dev_for_baseline.iob2")
test_sents, test_labels = read_universal_NER("train_dev_test_sets/test_for_baseline.iob2")

In [192]:
# Helper function to process data
def preprocess_data(sents, labels):
    # Flatten the list of lists
    flat_data = [word for sublist in sents for word in sublist]
    flat_labels = [label for sublist in labels for label in sublist]

    # Join the words to form a single string
    text = ' '.join(flat_data)

    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Split the sentences into lists of tokens
    split_sentences = [sentence.split() for sentence in sentences]

    # Align labels to the split sentences
    split_labels = []
    label_index = 0
    for sentence in split_sentences:
        sentence_labels = []
        for _ in sentence:
            if label_index < len(flat_labels):
                sentence_labels.append(flat_labels[label_index])
                label_index += 1
        split_labels.append(sentence_labels)
    
    return split_sentences, split_labels

In [254]:
processed_train_sents, processed_train_labels = preprocess_data(train_sents, train_labels)
processed_dev_sents, processed_dev_labels = preprocess_data(dev_sents, dev_labels)
processed_test_sents, processed_test_labels = preprocess_data(test_sents, test_labels)

In [262]:

print(processed_test_sents[0])
print(processed_test_labels[0])

# for tok, label in zip(processed_train_sents[0], processed_train_labels[0]):
#     print(tok, label)
    

['[CLS]', 'A', 'year', 'is', 'an', 'awful', 'long', 'time', 'in', 'football', ',', 'and', 'that', 'has', 'certainly', 'proved', 'the', 'case', 'for', 'F', '##olar', '##in', 'Ba', '##log', '##un', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'I-PER', 'I-PER', 'O']


In [197]:
# Label Mappng
label2id = {
 'O': 0,
 'B-LOC': 1,
 'I-LOC': 2,
 'B-PER': 3,
 'I-PER': 4,
 'B-ORG': 5,
 'I-ORG': 6,
 'B-TIM': 7,
 'I-TIM': 8,
 'B-MON': 9,
 'I-MON': 10,
 'B-DAT': 11,
 'I-DAT': 12,
 'B-PCT': 13,
 'I-PCT': 14,
}

id2label = {v: k for k, v in label2id.items()}

In [198]:
label_names = [key for key in label2id.keys()]

In [199]:
def labels_map2id(labels, mapping):
    label_ids = []
    for sent in labels:
        label_id = [mapping.get(label, label) for label in sent]
        label_ids.append(label_id)
    return label_ids

In [274]:
train_label_ids = labels_map2id(processed_train_labels, label2id)
dev_label_ids = labels_map2id(processed_dev_labels, label2id)
test_label_ids = labels_map2id(processed_test_labels, label2id)

In [278]:
# Create datasets
train_dataset = Dataset.from_dict({"tokens": processed_train_sents, "ner_tags": train_label_ids})
dev_dataset = Dataset.from_dict({"tokens": processed_dev_sents, "ner_tags": dev_label_ids})
test_dataset = Dataset.from_dict({"tokens": processed_test_sents, "ner_tags": test_label_ids})

print(test_dataset[0])

{'tokens': ['[CLS]', 'A', 'year', 'is', 'an', 'awful', 'long', 'time', 'in', 'football', ',', 'and', 'that', 'has', 'certainly', 'proved', 'the', 'case', 'for', 'F', '##olar', '##in', 'Ba', '##log', '##un', '.'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 0]}


In [280]:
raw_datasets = DatasetDict({
    "train": train_dataset,
    "dev": dev_dataset,
    "test": test_dataset
})

# Print to verify
print(raw_datasets['test'][0])

{'tokens': ['[CLS]', 'A', 'year', 'is', 'an', 'awful', 'long', 'time', 'in', 'football', ',', 'and', 'that', 'has', 'certainly', 'proved', 'the', 'case', 'for', 'F', '##olar', '##in', 'Ba', '##log', '##un', '.'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 0]}


In [203]:
model_name = 'distilbert-base-cased'
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

/Users/chris/myenv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [204]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [205]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs


### ÆNDRER ????

In [206]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=["tokens", "ner_tags"]
)

Map:   0%|          | 0/1586 [00:00<?, ? examples/s]

Map:   0%|          | 0/447 [00:00<?, ? examples/s]

Map:   0%|          | 0/479 [00:00<?, ? examples/s]

In [281]:
print(tokenized_datasets["test"][0])

{'input_ids': [101, 101, 138, 1214, 1110, 1126, 9684, 1263, 1159, 1107, 1709, 117, 1105, 1115, 1144, 4664, 4132, 1103, 1692, 1111, 143, 108, 108, 184, 5815, 108, 108, 1107, 18757, 108, 108, 9366, 108, 108, 8362, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, -100]}


In [208]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
metric = evaluate.load("seqeval")

In [282]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [283]:
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    id2label=id2label,
    label2id=label2id,
)

/Users/chris/myenv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [149]:
notebook_login()

In [211]:
args = TrainingArguments(
    "bert-finetuned-ner-football",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [299]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

  0%|          | 0/597 [00:00<?, ?it/s]

ValueError: expected sequence of length 8 at dim 1 (got 54)

In [213]:
trainer.push_to_hub(commit_message="Training complete")

CommitInfo(commit_url='https://huggingface.co/ChristianSneffeFleischer/bert-finetuned-ner-football/commit/33fc30f4dd819da2f9c54b8f68a80aef1415e993', commit_message='Training complete', commit_description='', oid='33fc30f4dd819da2f9c54b8f68a80aef1415e993', pr_url=None, pr_revision=None, pr_num=None)

In [284]:
eval_results = trainer.evaluate()
print(eval_results)

  0%|          | 0/56 [00:00<?, ?it/s]

{'eval_loss': 0.14448639750480652, 'eval_precision': 0.8577405857740585, 'eval_recall': 0.9017595307917888, 'eval_f1': 0.8791994281629736, 'eval_accuracy': 0.9625079310145931, 'eval_runtime': 3.0007, 'eval_samples_per_second': 148.964, 'eval_steps_per_second': 18.662, 'epoch': 3.0}


/Users/chris/myenv/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [248]:
tokenized_datasets["test"]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 479
})

In [266]:
trainer.evaluate(tokenized_datasets["test"])

  0%|          | 0/60 [00:00<?, ?it/s]

/Users/chris/myenv/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.12803532183170319,
 'eval_precision': 0.8722860791826309,
 'eval_recall': 0.9143239625167336,
 'eval_f1': 0.8928104575163399,
 'eval_accuracy': 0.9696969696969697,
 'eval_runtime': 3.0214,
 'eval_samples_per_second': 158.538,
 'eval_steps_per_second': 19.859,
 'epoch': 3.0}

In [297]:
# Make predictions on the test set
predictions, labels, metrics = trainer.predict(tokenized_datasets["test"])

  0%|          | 0/60 [00:00<?, ?it/s]

/Users/chris/myenv/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [294]:
def compute_preds(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels

In [298]:
compute_preds((predictions, labels))

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'I-PER',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'I-ORG',
  'I-ORG',
  'I-ORG',
  'I-ORG',
  'I-ORG',
  'O',
  'B-ORG',
  'I-ORG',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O',
  'B-ORG',
  'I-ORG',
  'I-ORG',
  'I-ORG',
  'I-ORG',
  'I-ORG',
  'I-ORG',
  'I-ORG',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'I-ORG',
  'I-ORG',
  'I-ORG',
  'I-ORG',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',

In [286]:
# Convert predictions to label indices
predictions = np.argmax(predictions, axis=-1)

In [287]:
print(predictions[0])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 0
 4 4 0 0 3 3 3 3 3 3 0 0 5 0 3 0 3 0 0 0 0 0 3 0 0 0 0 0 0 0 3 3 3 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 0 0 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [240]:
# # Ensure predictions is a list of lists
# if isinstance(predictions, np.ndarray):
#     if predictions.ndim == 1:
#         predictions = predictions.tolist()
#     else:
#         predictions = [pred.tolist() for pred in predictions]

# # Convert label indices to label names
# true_predictions = [
#     [id2label[p] for p in prediction]
#     for prediction in predictions
]

In [288]:
# Function to write predictions
def write_iob2_predictions(test_sents, predictions, output_file):
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for i, (sent, prediction) in enumerate(zip(test_sents, predictions)):
            outfile.write(f"# sent_id = test-{i+1:04d}\n")
            outfile.write(f"# text = {' '.join(sent)}\n")
            for j, (word, label) in enumerate(zip(sent, prediction)):
                outfile.write(f"{j+1}\t{word}\t{label}\t-\t-\n")
            outfile.write("\n")

In [289]:
# Write the shifted predictions to a file
write_iob2_predictions(processed_test_sents, true_predictions, 'baseline_preds/baseline_preds_football.iob2')

________________________________________________________________

In [3]:
# Testing

model_checkpoint = "ChristianSneffeFleischer/bert-finetuned-ner"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("In Argentina , beef is revered , respected , and praised .")

/Users/chris/myenv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[{'entity_group': 'LOC',
  'score': 0.99479514,
  'word': 'Argentina',
  'start': 3,
  'end': 12}]